In [ ]:
%cd '/content/drive/MyDrive/UCO/projects/epilepsy'

/content/drive/MyDrive/UCO/projects/epilepsy


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.0 MB/s eta 0:00:00


In [ ]:
# prompt: import data_prep.py in the directory

import sys
sys.path.append('./') # Assuming data_prep.py is in the same directory
print(sys.path)
from torch_geometric.data import HeteroData
from model import HeteroGAT3, HeteroGAT4
from data_prep import heterdataPrep, connect_diagnosis_edges
from util import balance_homodata_underSample, load_homodata, train_test_split_homodata


['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor', '/root/.ipython', './']


In [ ]:
data = load_homodata('/content/drive/MyDrive/UCO/projects/epilepsy/EPI_heterodata_diag200_first.h5')

In [ ]:
data

HeteroData(
  x=[1701885, 4],
  edge_index=[2, 20433323],
  edge_attr=[20433323, 1],
  y=[1701685]
)

In [ ]:
# data_balanced = balance_homodata_underSample(data, offset = 200)

class counts =  tensor([1598998,  102687])
minority_class:  tensor(1)
majority_class:  tensor(0)
majority_indices:  tensor([    201,     202,     203,  ..., 1701882, 1701883, 1701884])
majority_indices_size:  torch.Size([1598998])
selected_majority_indices:  tensor([1341560,  297379, 1147774,  ..., 1461296,  668533, 1104865])


In [ ]:
# data1 = heterdataPrep(data_balanced, offset = 200, agg = False)

/content/drive/MyDrive/UCO/projects/epilepsy/data_prep.py:15: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  out['diagnosis'].x =torch.range(0, offset-1).unsqueeze(1) # the index of diagnosis matters. It is a feature of the diagnosis node


In [ ]:
# Check if GPU is available
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [ ]:
train_data, test_data = train_test_split_homodata(data,0.8, 200)

In [ ]:
train_data = balance_homodata_underSample(train_data, offset = 200)
test_data_balanced = balance_homodata_underSample(test_data, offset = 200)

class counts =  tensor([1279394,   81954])
minority_class:  tensor(1)
majority_class:  tensor(0)
class counts =  tensor([319604,  20733])
minority_class:  tensor(1)
majority_class:  tensor(0)


In [ ]:
from torch_geometric.utils import subgraph
def dataloader_subgraph_homo(data, batch_size, preserve):
  # preserve is the number of samples at first to retain in each batch
  datalist = []
  num_batches = (data.x.shape[0] - preserve) // batch_size
  print('num_batches',num_batches)
  for i in range(0,num_batches):
    indices = torch.cat([torch.arange(0,preserve),torch.arange(preserve + i * batch_size, preserve + (i + 1) * batch_size, dtype=int)])
    subgraph_edge_index, subgraph_edge_attr = subgraph(indices, data.edge_index, data.edge_attr, relabel_nodes= True)
    # print('ran1')
    subgraphed = HeteroData()
    subgraphed.x = data.x[indices]
    subgraphed.edge_index = subgraph_edge_index
    subgraphed.edge_attr = subgraph_edge_attr
    subgraphed.y = data.y[preserve + i * batch_size:preserve + (i + 1) * batch_size]
    datalist.append(subgraphed)

  return datalist

In [ ]:
train_data_batched = dataloader_subgraph_homo(train_data, 3000, 200)

num_batches 54


In [ ]:
GAT2 = HeteroGAT4(input_features=3, edge_features=1, hidden_channels=32, convhidden_channels=32, out_channels=2, num_heads=6, metadata=train_data_d2d.metadata(), num_layers=4).to(device)  # 2 output channels for binary classification
optimizer = torch.optim.Adam(GAT2.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# Define optimizer and loss function
import torch


# Training loop (example)
for epoch in range(100):
  for batch in train_data_batched:
    GAT2.train()
    optimizer.zero_grad()
    batch = heterdataPrep(batch, offset = 200, agg = False)
    batch_data_d2d = connect_diagnosis_edges(batch).to(device)
    print(batch_data_d2d)
    out = GAT2(batch_data_d2d.x_dict, batch_data_d2d.edge_index_dict, batch_data_d2d.edge_attr_dict)
    loss = criterion(out, batch_data_d2d['person'].y)
    loss.backward()
    optimizer.step()

    print(f"Epoch: {epoch}, Loss: {loss.item()}")

HeteroData(
  diagnosis={ x=[200, 1] },
  person={
    x=[3000, 3],
    y=[3000],
  },
  (person, to, diagnosis)={
    edge_index=[2, 40147],
    edge_attr=[40147, 1],
  },
  (diagnosis, to, person)={ edge_index=[2, 40147] },
  (diagnosis, diag_conn, diagnosis)={ edge_index=[2, 19900] }
)


IndexError: Found negative indices in 'edge_index' (got -600). Please ensure that all indices in 'edge_index' point to valid indices in the interval [0, 3000) in your node feature matrix and try again.

In [ ]:
train_data = heterdataPrep(train_data, offset = 200, agg = False)
test_data = heterdataPrep(test_data, offset = 200, agg = False)
test_data_balanced = heterdataPrep(test_data_balanced, offset = 200, agg = False)

In [ ]:
train_data

HeteroData(
  diagnosis={ x=[200, 1] },
  person={
    x=[164154, 3],
    y=[164154],
  },
  (person, to, diagnosis)={
    edge_index=[2, 2148521],
    edge_attr=[2148521, 1],
  },
  (diagnosis, to, person)={ edge_index=[2, 2148521] }
)

In [ ]:
train_data_d2d = connect_diagnosis_edges(train_data)
test_data_d2d = connect_diagnosis_edges(test_data)
test_data_balanced_d2d = connect_diagnosis_edges(test_data_balanced)

In [ ]:
train_data_d2d

HeteroData(
  diagnosis={ x=[200, 1] },
  person={
    x=[164154, 3],
    y=[164154],
  },
  (person, to, diagnosis)={
    edge_index=[2, 2148521],
    edge_attr=[2148521, 1],
  },
  (diagnosis, to, person)={ edge_index=[2, 2148521] },
  (diagnosis, diag_conn, diagnosis)={ edge_index=[2, 19900] }
)

In [ ]:
train_data, test_data, test_data_balanced = train_data_d2d.to(device), test_data_d2d.to(device), test_data_balanced_d2d.to(device)

In [ ]:
GAT2 = HeteroGAT4(input_features=3, edge_features=1, hidden_channels=32, convhidden_channels=32, out_channels=2, num_heads=6, metadata=train_data_d2d.metadata(), num_layers=4).to(device)  # 2 output channels for binary classification
optimizer = torch.optim.Adam(GAT2.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()


In [ ]:
GAT2

In [ ]:
# Define optimizer and loss function
import torch


# Training loop (example)
for epoch in range(100):
    GAT2.train()
    optimizer.zero_grad()
    out = GAT2(train_data.x_dict, train_data.edge_index_dict, train_data.edge_attr_dict)
    loss = criterion(out, train_data_d2d['person'].y)
    loss.backward()
    optimizer.step()

    print(f"Epoch: {epoch}, Loss: {loss.item()}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 264.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 43.06 MiB is free. Process 27627 has 14.70 GiB memory in use. Of the allocated memory 14.07 GiB is allocated by PyTorch, and 515.13 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# data_test = train_data
data_test = test_data_balanced

In [ ]:
# prompt: get a confusion matrix for the trained model

# Evaluate the model
GAT2.eval()
with torch.no_grad():
  out = GAT2(data_test.x_dict, data_test.edge_index_dict, data_test.edge_attr_dict)
  # print(out)
  _, predicted = torch.max(out, dim=1)  # Predictions for persons only
  # predicted = out > 0.5  # Predictions for persons only

# Get the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(data_test['person'].y.cpu(), predicted.cpu())
print(cm)

[[ 8801 11477]
 [ 1924 18354]]


In [ ]:
# prompt: get a classification report

from sklearn.metrics import classification_report
print(classification_report(data_test['person'].y.cpu(), predicted.cpu()))

              precision    recall  f1-score   support

           0       0.82      0.43      0.57     20278
           1       0.62      0.91      0.73     20278

    accuracy                           0.67     40556
   macro avg       0.72      0.67      0.65     40556
weighted avg       0.72      0.67      0.65     40556



In [ ]:
# prompt: calculate auc

from sklearn.metrics import roc_auc_score

# Assuming 'out' contains the raw output scores from your model
# and 'data_test['person'].y' contains the true labels.

# Extract probabilities for the positive class (class 1)
probabilities = torch.softmax(out, dim=1)[:, 1].cpu().numpy()

# Calculate the AUC
auc = roc_auc_score(data_test['person'].y.cpu(), probabilities)
print(f"AUC: {auc}")

AUC: 0.7329903387688719


In [ ]:
# prompt: calculate PRAUC

from sklearn.metrics import precision_recall_curve, auc

# ... (your existing code)

# Assuming 'out' contains the raw output scores from your model
# and 'data_test['person'].y' contains the true labels.

# Extract probabilities for the positive class (class 1)
probabilities = torch.softmax(out, dim=1)[:, 1].cpu().numpy()

# Calculate precision-recall curve
precision, recall, thresholds = precision_recall_curve(data_test['person'].y.cpu(), probabilities)

# Calculate PRAUC
prauc = auc(recall, precision)
print(f"PRAUC: {prauc}")

PRAUC: 0.691939556480019
